In [1]:
import pandas as pd
import pandas_datareader
from pandas_datareader import get_data_yahoo as yh

stock_count=5
asset_count=2
ma_short=15
ma_long=50

In [2]:
myassetlist = [
    'TSLA',
    'GOOG',
    'AAPL',
    'SPY',
    'XLI'
]

In [3]:
df = yh(myassetlist)['Close'] # Get close prices

# Get 20 day returns and rank
ranked_assets = pd.DataFrame(df.pct_change(20).mean().sort_values(ascending = False)) # Ascending to descending returns
ranked_assets # full assets

,0
Symbols,
TSLA,0.067728
AAPL,0.031265
GOOG,0.016994
SPY,0.012289
XLI,0.011304


In [16]:
#moving average short/long term
ranked_ma_assets = pd.DataFrame((df.tail(ma_short).mean()-df.tail(ma_long).mean()).sort_values(ascending = False))
ranked_ma_to_trade = pd.concat([ranked_ma_assets.head(asset_count),ranked_ma_assets.tail(asset_count)])
ranked_ma_to_trade['Direction']= -1
for row in ranked_ma_to_trade.head(asset_count):
    ranked_ma_to_trade.head(asset_count)['Direction']= 1
ranked_ma_to_trade.columns = ['MA_strategy','Direction']
ranked_ma_to_trade

<ipython-input-16-2b37adc6b31a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranked_ma_to_trade.head(asset_count)['Direction']= 1


,MA_strategy,Direction
Symbols,,
TSLA,166.601596,1
GOOG,11.193807,1
AAPL,5.295600,-1
XLI,0.818001,-1


In [17]:
long = ranked_assets[0:asset_count] # top 2 assets
long['Direction'] = 1
long.columns = ['20_day_returns','Direction']
long

<ipython-input-17-0c0a9a381f0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long['Direction'] = 1


,20_day_returns,Direction
Symbols,,
TSLA,0.067728,1
AAPL,0.031265,1


In [18]:
short = ranked_assets[-asset_count:] # bottom 2 assets
short['Direction'] = -1
short.columns = ['20_day_returns','Direction']
short

<ipython-input-18-5132bb91ccc5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short['Direction'] = -1


,20_day_returns,Direction
Symbols,,
SPY,0.012289,-1
XLI,0.011304,-1


In [19]:
assets_to_trade = pd.concat([long,short])

In [20]:
assets_to_trade

,20_day_returns,Direction
Symbols,,
TSLA,0.067728,1
AAPL,0.031265,1
SPY,0.012289,-1
XLI,0.011304,-1


# Combine strategies

In [ ]:
combined = pd.DataFrame(pd.concat([assets_to_trade,ranked_ma_to_trade])['Direction'])
combined = combined.reset_index()
combined = combined.groupby('Symbols').sum()

In [49]:
combined

,Direction
Symbols,
AAPL,0
GOOG,1
SPY,-1
TSLA,2
XLI,-2
